In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from google.colab import files
from matplotlib import pyplot as plt

In [2]:
files.upload()  # Upload zip file
!unzip Images.zip


Saving Images.zip to Images.zip
Archive:  Images.zip
   creating: Images/
  inflating: Images/gettyimages-590778113-612x612.jpg  
  inflating: __MACOSX/Images/._gettyimages-590778113-612x612.jpg  
  inflating: Images/gettyimages-1156318629-612x612.jpg  
  inflating: __MACOSX/Images/._gettyimages-1156318629-612x612.jpg  
  inflating: Images/gettyimages-71154695-612x612.jpg  
  inflating: __MACOSX/Images/._gettyimages-71154695-612x612.jpg  
  inflating: Images/gettyimages-1224039863-612x612.jpg  
  inflating: __MACOSX/Images/._gettyimages-1224039863-612x612.jpg  
  inflating: Images/gettyimages-567353499-612x612.jpg  
  inflating: __MACOSX/Images/._gettyimages-567353499-612x612.jpg  
  inflating: Images/gettyimages-651424277-612x612.jpg  
  inflating: __MACOSX/Images/._gettyimages-651424277-612x612.jpg  
  inflating: Images/gettyimages-651424295-612x612.jpg  
  inflating: __MACOSX/Images/._gettyimages-651424295-612x612.jpg  
  inflating: Images/gettyimages-651424211-612x612.jpg  
  infla

In [3]:
path = 'Images' # path to the directory containing the images
results_path='Results'
os.makedirs(results_path, exist_ok=True)
image_list=[f.path for f in os.scandir(path)] # create an iterator for the directory

In [4]:
!pip install ultralytics
from ultralytics import YOLO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 13.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [5]:
def calculate_centroid_distances(customer_boxes, car_boxes):
    """Calculates the distances between customer and nearest car centroids.

    Args:
        customer_boxes (ndarray): Array of customer bounding boxes (xyxy format).
        car_boxes (ndarray): Array of car bounding boxes (xyxy format).

    Returns:
        tuple: A tuple containing two dictionaries:
            - centroid_distances: Dictionary mapping customer-car pair IDs to distances.
            - nearest_cars: Dictionary mapping customer IDs to their nearest car indices.
    """

    customer_centers = (customer_boxes[:, :2] + customer_boxes[:, 2:]) / 2
    car_centers = (car_boxes[:, :2] + car_boxes[:, 2:]) / 2

    centroid_distances = {}
    nearest_cars = {}

    for i, customer_center in enumerate(customer_centers):
        distances = np.linalg.norm(car_centers - customer_center, axis=1)
        nearest_car_idx = np.argmin(distances)
        nearest_car_distance = distances[nearest_car_idx]
        centroid_distances[f"customer_{i+1}-car_{nearest_car_idx+1}"] = nearest_car_distance
        nearest_cars[f"customer_{i+1}"] = nearest_car_idx

    return centroid_distances, nearest_cars



def draw_boxes_and_lines(image, customer_boxes, car_boxes, centroid_distances, nearest_cars):
  for i, box in enumerate(customer_boxes):
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, f"customer_{i+1}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    if f"customer_{i+1}" in nearest_cars:
      car_idx = nearest_cars[f"customer_{i+1}"]
      car_box = car_boxes[car_idx]
      cx1, cy1, cx2, cy2 = map(int, car_box)
      customer_center = ((x1 + x2) // 2, (y1 + y2) // 2)
      car_center = ((cx1 + cx2) // 2, (cy1 + cy2) // 2)
      cv2.line(image, customer_center, car_center, (0, 255, 255), 2)
      distance = centroid_distances[f"customer_{i+1}-car_{car_idx+1}"]
      mid_point = ((customer_center[0] + car_center[0]) // 2, (customer_center[1] + car_center[1]) // 2)
      cv2.putText(image, f"{distance:.2f}px", mid_point, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

  for i, box in enumerate(car_boxes):
    x1, y1, x2, y2 = map(int, box)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.putText(image, f"car_{i+1}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

  return image



In [6]:
# Initialize the model
model = YOLO("yolov8n.pt")

# Predict the results
results = model.predict(image_list, save=False, conf=0.5)

data = []
images_without_cars = 0
total_images = len(results)

# Iterate through the results
for result in results:
  img_path = result.path
  image = cv2.imread(img_path)
  boxes = result.boxes.xyxy.cpu().numpy()
  classes = result.boxes.cls.cpu().numpy()

  num_cars = (classes == 2).sum()
  num_customers = (classes == 0).sum()

  if num_cars == 0:
    images_without_cars += 1
    centroid_distances = {}  # Empty dictionary if no cars
  else:
    customer_boxes = boxes[classes == 0]
    car_boxes = boxes[classes == 2]
    # Calculate centroid distances
    centroid_distances, nearest_cars = calculate_centroid_distances(customer_boxes, car_boxes)
    image_with_annotations = draw_boxes_and_lines(image.copy(), customer_boxes, car_boxes, centroid_distances, nearest_cars)
    cv2.imwrite(os.path.join(results_path, os.path.basename(img_path)), image_with_annotations)

  data.append({
    "Image Path": img_path,
    "Number of Customers": num_customers,
    "Distances to Nearest Car Center": centroid_distances,
    "Number of Cars": num_cars
  })

# Create the DataFrame
df = pd.DataFrame(data)

# Calculate and print the percentage of images without cars
percentage_no_cars = (images_without_cars / total_images) * 100
print(f"\nPercentage of images without cars: {percentage_no_cars:.2f}%")

# Save the DataFrame to a CSV file
df.to_csv('results.csv', index=False)


100%|██████████| 6.23M/6.23M [00:00<00:00, 287MB/s]



0: 640x640 3 persons, 1 tie, 290.2ms
1: 640x640 1 person, 1 car, 1 truck, 290.2ms
2: 640x640 4 persons, 1 tie, 290.2ms
3: 640x640 3 persons, 2 cars, 290.2ms
4: 640x640 3 persons, 7 cars, 1 potted plant, 290.2ms
5: 640x640 3 persons, 2 cars, 290.2ms
6: 640x640 30 cars, 290.2ms
7: 640x640 2 persons, 1 cell phone, 290.2ms
8: 640x640 3 persons, 1 chair, 290.2ms
9: 640x640 1 person, 2 cars, 1 handbag, 290.2ms
10: 640x640 3 cars, 1 truck, 290.2ms
11: 640x640 2 persons, 3 cars, 1 truck, 290.2ms
12: 640x640 2 cars, 290.2ms
13: 640x640 2 persons, 5 cars, 290.2ms
14: 640x640 2 persons, 1 car, 290.2ms
15: 640x640 1 person, 2 cars, 290.2ms
16: 640x640 1 person, 1 airplane, 290.2ms
17: 640x640 2 persons, 2 cars, 290.2ms
18: 640x640 3 persons, 1 car, 290.2ms
19: 640x640 1 person, 3 cars, 290.2ms
Speed: 14.7ms preprocess, 290.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

Percentage of images without cars: 25.00%
